In [1]:
import sys
import os

# Adjust the path to point to my project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path

In [3]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )
        return data_ingestion_config

In [5]:
import urllib.request as request
from src.datascience import logger

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionconfig):
        self.config = config
    
    def download_file(self):
        """Download the CSV file from the source URL"""
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")
    
    def initiate_data_ingestion(self):
        """Main method to initiate the data ingestion process"""
        logger.info("Entered the data ingestion method or component")
        
        try:
            # Download the CSV file
            self.download_file()
            logger.info("Data ingestion completed successfully")
            
            return (
                self.config.local_data_file,
            )
            
        except Exception as e:
            logger.error(f"Error occurred during data ingestion: {e}")
            raise e

In [7]:
# Initialize the configuration
config_manager = ConfigurationManager()
data_ingestion_config = config_manager.get_data_ingestion_config()

# Create DataIngestion instance
data_ingestion = DataIngestion(data_ingestion_config)

# Initiate data ingestion
local_data_file = data_ingestion.initiate_data_ingestion()
print(f"Data downloaded to: {local_data_file}")

[2025-06-25 19:07:39,371: INFO: common: yaml file: ../config/config.yaml loaded successfully]
[2025-06-25 19:07:39,372: INFO: common: yaml file: ../params.yaml loaded successfully]
[2025-06-25 19:07:39,372: INFO: common: yaml file: ../schema.yaml loaded successfully]
[2025-06-25 19:07:39,373: INFO: common: created directory at: artifacts]
[2025-06-25 19:07:39,373: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-25 19:07:39,374: INFO: 470975429: Entered the data ingestion method or component]
[2025-06-25 19:07:39,780: INFO: 470975429: artifacts/data_ingestion/winequality-white.csv downloaded! with following info: 
Connection: close
Content-Length: 264426
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "b59f798118e63a6fe2399f503b5411c00a46fb511fc74357511ef218d79f7186"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: de